# Lab - Recommender System


In [2]:
import pandas as pd

In [3]:
movies = pd.read_csv("../../Data/ml-latest/movies.csv")
ratings = pd.read_csv("../../Data/ml-latest/ratings.csv")

In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


In [6]:
ratings["movieId"].nunique()

53889

In [7]:
most_ratings = ratings["rating"].groupby(ratings["movieId"]).count()
most_ratings = pd.DataFrame(most_ratings.sort_values(ascending=False).head(10)).reset_index()
most_ratings = most_ratings.rename({"rating":"Number of Ratings"}, axis=1)
most_ratings

,movieId,Number of Ratings
0,318,97999
1,356,97040
2,296,92406
3,593,87899
4,2571,84545
5,260,81815
6,480,76451
7,527,71516
8,110,68803
9,1,68469


In [8]:
movies_most_ratings = pd.merge(most_ratings, 
                            movies.loc[movies["movieId"].isin(most_ratings["movieId"])], 
                            on="movieId"
                            )
movies_most_ratings

,movieId,Number of Ratings,title,genres
0,318,97999,"Shawshank Redemption, The (1994)",Crime|Drama
1,356,97040,Forrest Gump (1994),Comedy|Drama|Romance|War
2,296,92406,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
3,593,87899,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller
4,2571,84545,"Matrix, The (1999)",Action|Sci-Fi|Thriller
5,260,81815,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi
6,480,76451,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller
7,527,71516,Schindler's List (1993),Drama|War
8,110,68803,Braveheart (1995),Action|Drama|War
9,1,68469,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [9]:
rating_of_most_rated = ratings.loc[ratings["movieId"].isin(most_ratings["movieId"])]
rating_of_most_rated = rating_of_most_rated["rating"].groupby(rating_of_most_rated["movieId"]).mean()


In [10]:
movies_most_rated = pd.merge(movies_most_ratings, 
                            pd.DataFrame({"Mean Rating":rating_of_most_rated}).reset_index(), 
                            on="movieId")

In [86]:
year_of_movies = movies["title"].str.extract(r"(\(\d{4}\))")
year_of_movies = year_of_movies[0].str.extract(r"(\d{4})").rename({0:"Year"}, axis=1)
year_of_movies = pd.DataFrame(year_of_movies["Year"].dropna().astype(int))
year_of_movies

,Year
0,1995
1,1995
2,1995
3,1995
4,1995
...,...
58093,1946
58094,2011
58095,2016
58096,2017


In [93]:
movies_per_year = year_of_movies["Year"].unique()
movies_per_year

array([1995, 1994, 1996, 1976, 1992, 1988, 1967, 1993, 1964, 1977, 1965,
       1982, 1985, 1990, 1991, 1989, 1937, 1940, 1969, 1981, 1973, 1970,
       1960, 1955, 1959, 1968, 1980, 1975, 1986, 1948, 1943, 1950, 1946,
       1987, 1997, 1974, 1956, 1958, 1949, 1972, 1998, 1933, 1952, 1951,
       1957, 1961, 1954, 1934, 1944, 1963, 1942, 1941, 1953, 1939, 1947,
       1945, 1938, 1935, 1936, 1926, 1932, 1979, 1971, 1978, 1966, 1962,
       1983, 1984, 1931, 1922, 1999, 1927, 1929, 1930, 1928, 1925, 1914,
       2000, 1919, 1923, 1920, 1918, 1921, 2001, 1924, 2002, 2003, 1915,
       2004, 1916, 1917, 2005, 2006, 1902, 1903, 2007, 2008, 2009, 1912,
       2010, 1913, 2011, 1898, 1899, 1894, 2012, 1909, 1910, 1901, 1893,
       2013, 1896, 2014, 2015, 1895, 1911, 1900, 2016, 2017, 2018, 1905,
       1904, 1891, 1892, 1908, 1897, 1887, 1888, 1890, 1878, 1874, 1907,
       1906, 1883])

In [65]:
test = movies_most_rated["title"].str.extract(r"(\d{4})").astype(int).rename({0:"Year"}, axis=1)
test

,Year
0,1994
1,1994
2,1994
3,1991
4,1999
5,1977
6,1993
7,1993
8,1995
9,1995
